# Sample Dataset: User-Song Interactions


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
data = {
    "user_id": [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5],
    "song_id": [101, 102, 103, 101, 104, 105, 102, 106, 107, 103, 104, 108, 105, 107, 109],
    "rating": [5, 4, 3, 5, 3, 4, 4, 3, 5, 2, 5, 4, 3, 4, 5]
}

df = pd.DataFrame(data)
df

,user_id,song_id,rating
0,1,101,5
1,1,102,4
2,1,103,3
3,2,101,5
4,2,104,3
5,2,105,4
6,3,102,4
7,3,106,3
8,3,107,5
9,4,103,2


# Pre-processing

In [3]:
# Encode user_id and song_id to consecutive integers
user_mapping = {user_id: idx for idx, user_id in enumerate(df["user_id"].unique())}
song_mapping = {song_id: idx for idx, song_id in enumerate(df["song_id"].unique())}

df["user_id"] = df["user_id"].map(user_mapping)
df["song_id"] = df["song_id"].map(song_mapping)

num_users = len(user_mapping)
num_songs = len(song_mapping)

print(df)
print(num_songs)

    user_id  song_id  rating
0         0        0       5
1         0        1       4
2         0        2       3
3         1        0       5
4         1        3       3
5         1        4       4
6         2        1       4
7         2        5       3
8         2        6       5
9         3        2       2
10        3        3       5
11        3        7       4
12        4        4       3
13        4        6       4
14        4        8       5
9


# PyTorch Dataset Class


In [5]:
class MusicDataset(Dataset):
    def __init__(self, dataframe):
        self.users = torch.tensor(dataframe["user_id"].values, dtype=torch.long)
        self.songs = torch.tensor(dataframe["song_id"].values, dtype=torch.long)
        self.ratings = torch.tensor(dataframe["rating"].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.songs[idx], self.ratings[idx]

# Create dataset and dataloader
dataset = MusicDataset(df)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

dataset[0]

(tensor(0), tensor(0), tensor(5.))

# Define the Neural Collaborative Filtering Model

In [6]:
class MusicRecommender(nn.Module):
    def __init__(self, num_users, num_songs, embedding_dim=16):
        super(MusicRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.song_embedding = nn.Embedding(num_songs, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, user, song):
        user_embedded = self.user_embedding(user)
        song_embedded = self.song_embedding(song)
        x = torch.cat([user_embedded, song_embedded], dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.squeeze()

# Instantiate model, loss function, and optimizer
model = MusicRecommender(num_users, num_songs)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Training the Model


In [7]:
def train_model(model, dataloader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for users, songs, ratings in dataloader:
            optimizer.zero_grad()
            predictions = model(users, songs)
            loss = criterion(predictions, ratings)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

train_model(model, dataloader, criterion, optimizer)


Epoch 1/20, Loss: 55.5755
Epoch 2/20, Loss: 13.8297
Epoch 3/20, Loss: 24.7205
Epoch 4/20, Loss: 4.3135
Epoch 5/20, Loss: 8.5426
Epoch 6/20, Loss: 8.7879
Epoch 7/20, Loss: 2.7527
Epoch 8/20, Loss: 2.9733
Epoch 9/20, Loss: 1.4321
Epoch 10/20, Loss: 0.6704
Epoch 11/20, Loss: 1.2347
Epoch 12/20, Loss: 0.3973
Epoch 13/20, Loss: 0.4254
Epoch 14/20, Loss: 0.1918
Epoch 15/20, Loss: 0.1501
Epoch 16/20, Loss: 0.1871
Epoch 17/20, Loss: 0.1077
Epoch 18/20, Loss: 0.1104
Epoch 19/20, Loss: 0.0154
Epoch 20/20, Loss: 0.0635


# Function to recommend songs for a given user

In [8]:
def recommend_songs(user_id, model, num_recommendations=5):
    model.eval()
    user_idx = user_mapping.get(user_id, None)
    if user_idx is None:
        raise ValueError("User not found in dataset")

    all_songs = torch.tensor(list(song_mapping.values()), dtype=torch.long)
    user_tensor = torch.tensor([user_idx] * len(all_songs), dtype=torch.long)

    with torch.no_grad():
        predictions = model(user_tensor, all_songs)

    top_song_indices = predictions.argsort(descending=True)[:num_recommendations]
    recommended_song_ids = [list(song_mapping.keys())[i] for i in top_song_indices.tolist()]

    return recommended_song_ids

# Example: Recommend songs for a user


In [12]:
user_to_recommend = 3
recommended_songs = recommend_songs(user_to_recommend, model)
print(f"Recommended songs for User {user_to_recommend}: {recommended_songs}")

Recommended songs for User 3: [107, 104, 109, 102, 101]
